In [2]:
import os
import pandas as pd
from matplotlib import pyplot as plt
from utils.time_utils import TimeUtil
%matplotlib inline
os.chdir("../")

In [42]:
def back_test_snowball(file, hedge_barrier=False):
    notional = 100000000
    df = pd.read_csv(file, index_col=0, parse_dates=True)
    df['$pv'] = df['pv'] * notional
    df['$delta'] = df['delta'] * notional * df['close']
    df['qty'] = df['$delta'] / df['close']
    df['cash'] = 0.
    for i in range(len(df.index)):
        today = df.index[i]
        if i == 0:
            df.loc[today, 'cash'] = df.loc[today, '$pv']-df.loc[today, '$delta']
        elif i >=0:
            yesterday = df.index[i-1]
            dt = TimeUtil.time_diff_natural_day(yesterday, today)
            if i < len(df.index) - 1:
                df.loc[today, 'cash'] = df.loc[yesterday, 'cash'] * (1 + 0.025 * dt) - (df.loc[today, 'qty'] - df.loc[yesterday, 'qty']) * df.loc[today, 'close']
            else:
                df.loc[today, 'cash'] = df.loc[yesterday, 'cash'] * (1 + 0.025 * dt) - df.loc[today, '$pv'] + df.loc[yesterday, 'qty'] * df.loc[today, 'close']
    return df['cash'].values[-1] / notional

In [43]:
frame = back_test_snowball("./000905_calc/flat_3m/2009-01-05.csv")

In [44]:
frame

-0.0212955343120669

In [45]:
file_str = "./000905_calc/{}/{}"

In [46]:
vol_scheme = 'flat_3m'

In [47]:
files = os.listdir("./000905_calc/flat_3m/")

In [48]:
result = {}
for vol_scheme in ['flat_3m', 'flat_12m']:
    result[vol_scheme] = {}
    for file_ in files:
        file = file_str.format(vol_scheme, file_)
        result[vol_scheme][file_.split('.')[0]] = back_test_snowball(file)

In [49]:
result

{'flat_3m': {'2009-01-05': -0.0212955343120669,
  '2009-02-02': 0.004056303225021735,
  '2009-03-02': -0.017067109865245633,
  '2009-04-01': -0.03183652681107093,
  '2009-05-04': -0.09013388507183578,
  '2009-06-01': 0.019783274614519626,
  '2009-07-01': 0.1246275459555104,
  '2009-08-03': 0.10860386220699167,
  '2009-09-01': 0.0033504766511636974,
  '2009-10-09': -0.021225430130952617,
  '2009-11-02': -0.031585700014624817,
  '2009-12-01': -0.048621050981176944,
  '2010-01-04': -0.04876487614672497,
  '2010-02-01': -0.10878325109748575,
  '2010-03-01': -0.0607729613997072,
  '2010-04-01': 0.11139156249204307,
  '2010-05-04': -0.017033631419971437,
  '2010-06-01': 0.00471515497440815,
  '2010-07-01': -0.007411483145169113,
  '2010-08-02': -0.04730993946701494,
  '2010-09-01': -0.03995042591629228,
  '2010-10-08': 0.030310314959039762,
  '2010-11-01': 0.0371267111741665,
  '2010-12-01': -0.028508003030259313,
  '2011-01-04': -0.004474825609478503,
  '2011-02-01': -0.06125568875955284,
 

In [50]:
result_frame = pd.DataFrame(result)

In [51]:
result_frame

,flat_3m,flat_12m
2009-01-05,-0.021296,-0.017765
2009-02-02,0.004056,-0.013477
2009-03-02,-0.017067,-0.033665
2009-04-01,-0.031837,-0.059284
2009-05-04,-0.090134,-0.059085
...,...,...
2020-12-01,-0.011383,-0.114715
2021-01-04,-0.006966,-0.103769
2021-02-01,-0.013683,-0.096969
2021-03-01,0.001369,-0.038086


In [52]:
result_frame.std()

flat_3m     0.108846
flat_12m    0.111887
dtype: float64

In [53]:
result_frame.mean()

flat_3m    -0.005605
flat_12m   -0.028050
dtype: float64